<a href="https://colab.research.google.com/github/Samitha-Nawarathna/GNN-for-Text-Analysis/blob/main/Model_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=2c4ccf3fb07c213c1ecd065161f4b81fee50d915d45dcd57a586f88da4f7524f
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [3]:
data_path = "/content/drive/MyDrive/Datasets/bbc-full-text-document-classification/processed/"

In [4]:
import torch
import torch_geometric
import torch.nn as nn
import torch_geometric.nn as gnn
from torch_geometric.nn import aggr
from torch_geometric.nn import MessagePassing
from torch.utils.data import Dataset
import glob

In [5]:
class Graph_Dataset(Dataset):
  def __init__(self,path):
    super().__init__()
    self.path = path
    self.path_list = self.getPaths()

  def getPaths(self):
    return glob.glob(self.path+'*.pt')

  def __len__(self):
    len(self.path_list)

  def __getitem__(self,idx):
    print(idx)
    if type(idx) == slice:
      graphs = []
      start ,step, stop = idx.start, idx.step, idx.stop
      if start == None:
        start =0
      if stop == None:
        stop = len(self.path_list)-1
      if step == None:
        step =1
      print(type(stop))
      for item in range(start, stop, step):
        graphs.append(torch.load(self.path_list[item]))
      return graphs
    return torch.load(self.path_list[idx])

In [6]:
dataset = Graph_Dataset(data_path)

In [11]:
class MLP(nn.Module):
  def __init__(self,input_size, output_size):
    super().__init__()
    self.p1 = nn.Linear(input_size, 100)
    self.relu = nn.ReLU()
    self.p2 = nn.Linear(100,50)
    self.p3 = nn.Linear(50,output_size)

  def forward(self,x):
    x = self.p1(x)
    x = self.relu(x)
    x = self.p2(x)
    x = self.relu(x)
    x = self.p3(x)
    return x

class GraphModel(nn.Module):
  def __init__(self,input_size, output_size):
    super().__init__()
    self.gconv1 = gnn.conv.GatedGraphConv(out_channels=50, num_layers=5)
    self.f1 = MLP(input_size,output_size)
    self.f2 = MLP(input_size,output_size)
    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()

  def forward(self,x,edge_index,edge_attr):
    x = self.gconv1(x, edge_index, edge_weight = edge_attr)
    x1 = self.f1(x)
    x2 = self.f2(x)
    x = x1*x2
    xg = (1/x.shape[0])*torch.sum(x,dim=0) #+ torch.max(x, dim=0).type(torch.tensor)
    return xg
    #x = self.attention(x)


In [18]:
def extractGraph(graph):
   x = graph.x.type(torch.float)
   edge_index = graph.edge_index.type(torch.int64)
   edge_attr = graph.edge_attr.type(torch.float)
   return x, edge_index, edge_attr
def defineModel():
  model = GraphModel()

In [ ]:
def getPrediction(graph, model, output_size):
  x, edge_index, edge_attr = extractGraph(graph)
  model(x, edge_index, edge_attr)

In [13]:
model = GraphModel()
graph = dataset[5]

5


In [25]:
class a():
  p = 1
  q = 2

5


In [17]:
a = a()
a.r

NameError: ignored

In [14]:
x = model(graph)

In [16]:
len(x)

25

In [26]:
x = graph.x.type(torch.float)
edge_index = graph.edge_index.type(torch.int64)
edge_attr = graph.edge_attr.type(torch.float)
Conv1 = gnn.conv.GatedGraphConv(out_channels=50, num_layers=5)
Conv1(x, edge_index, edge_weight = edge_attr)

tensor([[ 0.1061, -0.0944,  0.0388,  ...,  0.0833, -0.0373,  0.0507],
        [ 0.1462, -0.0857,  0.0540,  ...,  0.1088, -0.0316,  0.0025],
        [ 0.1264, -0.0466,  0.0621,  ...,  0.1802, -0.0798, -0.0228],
        ...,
        [ 0.1888, -0.0837, -0.0032,  ...,  0.1391, -0.0676, -0.0182],
        [ 0.1348, -0.0281, -0.0342,  ...,  0.1084, -0.0589, -0.0190],
        [ 0.2369, -0.0615, -0.0075,  ...,  0.0764, -0.1407, -0.1433]],
       grad_fn=<AddBackward0>)